In [ ]:
pip install sounddevice wavio speechrecognition scikit-learn


In [ ]:
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
import wavio
import speech_recognition as sr
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
# --- ML Model Setup ---
phrases = ["hey assistant", "hello", "hi bot", "wake up", "activate",
           "what time is it", "tell me a joke", "how are you", "stop listening", "good morning"]
labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(phrases, labels)

In [ ]:
# --- Core Functions ---
def record_audio(filename="recording.wav", duration=5, fs=44100):
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    wavio.write(filename, recording, fs, sampwidth=2)

In [ ]:
def transcribe_audio(filename="recording.wav"):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio)
    except:
        return ""

In [ ]:
def detect_trigger(text):
    if not text.strip():
        return "No speech detected."
    prediction = model.predict([text])[0]
    return "✅ Trigger word detected!" if prediction == 1 else "❎ Not a trigger word."


In [ ]:
def run_detection():
    record_audio()
    text = transcribe_audio()
    result = detect_trigger(text)
    messagebox.showinfo("Result", f"You said: {text}\n\n{result}")


In [ ]:
# --- GUI Setup ---
root = tk.Tk()
root.title("Trigger Word Detector")

tk.Label(root, text="🎤 Speak a command...").pack(pady=10)
tk.Button(root, text="Start Listening", command=run_detection, width=25).pack(pady=10)
tk.Label(root, text="(It will record for 5 seconds)").pack(pady=5)

root.mainloop()